As the title suggests, we will be using to fastai to achieve the best possible score with minimum lines of code.

We will also use **timm** for its pretrained models. Lets start by installing the latest version of both the libraries

In [ ]:
!pip install -U -q timm
!pip install -U -q fastai

Great, time to import both the libraries . . . 

In [ ]:
from timm import create_model
from fastai.vision.all import *

For experimentation, we will keeping the `config` at the top. 

Note: I prefer using dataclass, because dictionary syntax is very verbose. With dataclass, you can access the config values using `.` notation (that also means, tab auto-completion)

In [ ]:
from dataclasses import dataclass

@dataclass
class config:
    seed = 42
    
    # training
    lr = 5e-2
    epochs = 7
    freeze_epochs = 3
    arch = 'resnet200d_320'
    
    # data
    bs = 16
    fold = 2
    img_size = 456
    pre_img_size = 512

Before we write any code, lets set the seed for reproducibility. We will use fastai's `set_seed` function.

In [ ]:
set_seed(config.seed)

Next, we will initialize some path (& other) variables (for use throughout the notebook)

PS: I will be using my version for the dataset. I have resized all the images so that its much faster to load them into the RAM.

In [ ]:
path = Path('../input/ranzcr-clip-catheter-line-classification')
resized_path = Path('../input/rancer-resized-dataset/img_sz_512')
folds_path = Path('../input/ranzcr-folds')

In [ ]:
targets = ["ETT - Abnormal", "ETT - Borderline", "ETT - Normal", "NGT - Abnormal",
    "NGT - Borderline", "NGT - Incompletely Imaged", "NGT - Normal", "CVC - Abnormal",
    "CVC - Borderline", "CVC - Normal", "Swan Ganz Catheter Present"]

## Data

Enough prep-work! Lets dive in . . .

In [ ]:
df = pd.read_csv(folds_path/'train_folds.csv')
df.head()

Fastai's datablock API is amazing. Infinite flexibility and incredibly powerful. To use the datablock API, you need to define some functions.

In [ ]:
def get_x(x): return resized_path/'train'/(x['StudyInstanceUID']+'.jpg')
def get_y(y): return y[targets].tolist()
def splitter(df):
    trn_idx = df[df.kfold != config.fold].index.to_list()
    val_idx = df[df.kfold == config.fold].index.to_list()
    return trn_idx, val_idx

db = DataBlock(blocks= (ImageBlock, MultiCategoryBlock(encoded=True, vocab=targets)),
               get_x = get_x, 
               get_y = get_y,
               splitter = splitter,
               item_tfms = Resize(config.pre_img_size),
               batch_tfms = [*aug_transforms(size=config.img_size, min_scale=0.9), 
                             Normalize.from_stats(*imagenet_stats)]
               )

Don't worry a lot if the above code looks cryptic. You can read the [6th notebook (or chapter)](https://github.com/fastai/fastbook/blob/master/06_multicat.ipynb) in fastbook. It explains the topic in the most simplest way possible. And once you master datablock API, you will feel like a Ninja (trust me on this)! 

You are amazing! Now lets create our dataloaders, & then take a look at some images.

In [ ]:
dls = db.dataloaders(df, bs=config.bs)
dls.show_batch()

Looks great to me, What do you think?

We are done with data, time to build our deep learning model.

## Model

We will wrap timm's `create_model` function inside `create_timm_body` function (inspired from fastai's `create_body` function), to use fastai's esoteric features like differential learning rate.

Here is an amazing [colab notebook](https://colab.research.google.com/github/muellerzr/Practical-Deep-Learning-for-Coders-2.0/blob/master/Computer%20Vision/05_EfficientNet_and_Custom_Weights.ipynb) to help you better understand "how you can use custom models with fastai". 

In [ ]:
def create_timm_body(arch:str, pretrained=True, cut=None):
    model = create_model(arch, pretrained=pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

Once we have the *model body*, we will calculate the *output* shape and create the *model head* accordingly. 

You can easily calculate the output shape easily by using fastai's `num_features_model` function😉

Finally, we will use *kaimming initialization* to initialize the *model head* weights.

I know it sounds complex, but its just a few lines of code.

In [ ]:
body = create_timm_body(config.arch, pretrained=True)
nf = num_features_model(nn.Sequential(*body.children()))
head = create_head(nf, 11) 
model = nn.Sequential(body, head)
apply_init(model[1], nn.init.kaiming_normal_)

## Learner

Time to put everything together. Fastai has an awesome class for it, called `Learner`. 

In [ ]:
learner = Learner(dls, model, 
                  splitter=default_split, 
                  loss_func=BCEWithLogitsLossFlat(),
                  metrics=[accuracy_multi]).to_fp16()

And finally, lets train (technically, finetuning 🤯) our model.

In [ ]:
learner.fine_tune(config.epochs, freeze_epochs=config.freeze_epochs, base_lr=config.lr)

Amazing! Lets export the model so that we can deploy it to production 😂. Just kidding, we will (only) use it for inference.

In [ ]:
learner.export(f'{config.arch}_foldx{config.fold}.pkl')

Looking forward to inference? 